download annoatation from https://github.com/10XGenomics/single-cell-3prime-paper.
preprocess h5ad file from mtx, for example, code in https://github.com/imsb-uke/scGAN/blob/master/Misc/Datasets%20format%20conversion.ipynb can be used

In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
data = sc.read_h5ad('68kPBMCs.h5ad')
annotation = pd.read_csv('./68k_pbmc_barcodes_annotation.tsv',sep='\t')
annotation_dict = {item['barcodes']:item['celltype'] for item in annotation.to_dict('records')}
data.obs['barcodes'] = data.obs.index
data.obs['celltype'] = data.obs['barcodes'].map(annotation_dict)
sc.pp.filter_genes(data,min_cells=686)
sc.pp.filter_cells(data,min_genes=10)
sc.pp.normalize_per_cell(data,20000)
sc.pp.log1p(data)
sc.pp.highly_variable_genes(data,n_top_genes=1000)
data.X = np.exp(data.X.toarray())-1
data=data[:,data.var['highly_variable']]

In [ ]:
clustered = data.copy()
sc.pp.recipe_zheng17(clustered)
sc.pp.neighbors(clustered, n_pcs=50)
sc.tl.louvain(clustered, resolution=0.15)

In [ ]:
data.obs['louvain'] = clustered.obs['louvain']